In [1]:
from cwe2.database import Database
import nvdlib
import requests
from bs4 import BeautifulSoup
from pathlib import Path
from typing import List,Optional
import json
from pprint import pprint
from dataclasses import dataclass
from src.aggregator.cwe import get_mem_safe_cwes,data_dir

/home/Daedalus/Workspace/SecResearchProject/data


In [2]:
cwes = get_mem_safe_cwes()

In [ ]:
def get_cve_data(cve_id:str)->Optional[dict]:
    res={}
    cve_data=nvdlib.searchCVE(cveId=cve_id)
    for cve_entry in cve_data:
        if len(cve_entry) > 1:
            print(f"Warning: {cve_id} has more than one entry")
        
        for v in cve_entry[0]:
            if v==5:
                continue
            match v:
                case 'descriptions':
                    for desc_entry in cve_entry.__dict__[v]:
                        if desc_entry['lang']=='en':
                            res['description']=desc_entry['value']
                
            
        

In [10]:
tmp = nvdlib.searchCVE(cveId=cwes[0].associated_cves[0].name)[0]

In [63]:
tmp2=dict(tmp.__dict__)
tmp_res={}
for k,v in tmp2.items():
    match k:
        case 'descriptions':
            print(v)
            for desc_entry in v:
                if desc_entry.lang=='en':
                    print(desc_entry.value)
            tmp_res['description']=v
        case 'configurations':
            if len(v)>1:
                print("Warning: more than one configuration")
            v=v[0]
            if len(v.nodes)>1:
                print("Warning: more than one node")
            v=v.nodes[0]
            print(v.__dict__.keys())
            if len(v.__dict__.keys())>3:
                print("Warning: unhandled keys")
                print(v.__dict__.keys())
            tmp_conf={}
            tmp_conf["operator"]=str(v.operator)
            tmp_conf["negate"]=bool(v.negate)
            tmp_cpematch={}
            for x in v.cpeMatch:
                if len(x.__dict__.keys())>5:
                    print("Warning: unhandled keys")
                    print(x.__dict__.keys())
                
                tmp_cpematch['vulnerable']=bool(x.vulnerable)
                tmp_cpematch['criteria']=str(x.criteria)
                tmp_cpematch['versionStartIncluding']=str(x.versionStartIncluding)
                tmp_cpematch['versionEndExcluding']=str(x.versionEndExcluding)
                tmp_cpematch['matchCriteriaId']=str(x.matchCriteriaId)
            tmp_conf['cpeMatch']=tmp_cpematch
            
        case "v31vector":
            tmp_res["v31vector"]=v
        case "v31score":
            print(v)
            tmp_res["v31score"]=v
            
        case "v31severity":
            tmp_res["v31severity"]=v
        case "v31impactScore":
            tmp_res["v31impactScore"]=v
        case "v31exploitability":
            tmp_res["v31exploitability"]=v
        case "references":
            tmp_res["references"]=[]
            for ref in v:
                tmp_ref={}
                tmp_ref["url"]=ref.url
                tmp_ref["tags"]=[str(tag) for tag in ref.tags]
                tmp_res["references"].append(tmp_ref)
            
            break

        



        
        
    


[{'lang': 'en', 'value': 'On BIG-IP versions 16.0.x before 16.0.1.1, 15.1.x before 15.1.2.1, 14.1.x before 14.1.4, 13.1.x before 13.1.3.6, and 12.1.x before 12.1.5.3, undisclosed requests to a virtual server may be incorrectly handled by the Traffic Management Microkernel (TMM) URI normalization, which may trigger a buffer overflow, resulting in a DoS attack. In certain situations, it may theoretically allow bypass of URL based access control or remote code execution (RCE). Note: Software versions which have reached End of Software Development (EoSD) are not evaluated.'}, {'lang': 'es', 'value': 'En BIG-IP versiones 16.0.x anteriores a 16.0.1.1, versiones 15.1.x anteriores a 15.1.2.1, versiones 14.1.x anteriores a 14.1.4, versiones 13.1.x anteriores a 13.1.3.6 y versiones 12.1.x anteriores a 12.1.5.3, unas peticiones no reveladas a un servidor virtual pueden ser manejadas incorrectamente por la normalización del URI de Traffic Management Microkernel (TMM), lo que puede desencadenar un 

AttributeError: 'str' object has no attribute 'v31severity'

In [8]:
for cwe in cwes:
    if cwe.associated_cves is None:
        continue
    cwe_res = {}
    for cve in cwe.associated_cves:
        res= nvdlib.searchCVE(cveId=cve.name)
        #print(f"res: {json.loads(res[0].__dict__)}")
        # if len(res) > 1:
        #     cwe_res[cve.name] = [str(r.__dict__) for r in res]
        # else: 
        #     cwe_res[cve.name] = {v:getattr(res[0].__dict__,v) for v in res[0] if v != 5}
        for v in res[0]:
            if v == 5:
                continue
            match v:
                case 'descriptions':
                    for entry in res[0].__dict__[v]:
                        if entry.lang == 'en':
                            print(cve.description)
                            print(entry.value)
                case ""
                            
                    break
                
            print(f"field: {v}, type: {type(v)}, value: {res[0].__dict__[v]}")
            
        break
    # pprint(cwe_res)
    # data_dir.mkdir(exist_ok=True)
    # with open(data_dir/f"{cwe.id}.json","w") as f:
    #     json.dump(cwe_res,f,indent=4)
    break

field: id, type: <class 'str'>, value: CVE-2021-22991
field: sourceIdentifier, type: <class 'str'>, value: f5sirt@f5.com
field: published, type: <class 'str'>, value: 2021-03-31T18:15:14.787
field: lastModified, type: <class 'str'>, value: 2023-08-08T14:21:49.707
field: vulnStatus, type: <class 'str'>, value: Analyzed
field: cisaExploitAdd, type: <class 'str'>, value: 2022-01-18
field: cisaActionDue, type: <class 'str'>, value: 2022-02-01
field: cisaRequiredAction, type: <class 'str'>, value: Apply updates per vendor instructions.
field: cisaVulnerabilityName, type: <class 'str'>, value: F5 BIG-IP Traffic Management Microkernel Buffer Overflow
Incorrect URI normalization in application traffic product leads to buffer overflow, as exploited in the wild per CISA KEV. 
On BIG-IP versions 16.0.x before 16.0.1.1, 15.1.x before 15.1.2.1, 14.1.x before 14.1.4, 13.1.x before 13.1.3.6, and 12.1.x before 12.1.5.3, undisclosed requests to a virtual server may be incorrectly handled by the Traffic